<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefare_submission_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import math

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import numpy as np

RANDOM_SEED = 4

In [0]:
dataframe = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

dataframe.isna().sum(axis = 0)
#dataframe = dataframe.fillna(method='ffill')
dataframe = dataframe.dropna()

In [0]:
def preprocess_train_data(dataframe):
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  td = features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  td = get_trip_distance(td)
  td = get_X_distance(td)
  td = get_Y_distance(td)
  td = get_pick_XY(td)
  td = get_drop_XY(td)

  return td

def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]

  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

def set_time_of_the_day(cleaned_train_set):
  cleaned_train_set.head()
  newFrame = cleaned_train_set
  tt = newFrame

  d0_index = newFrame[(newFrame['pickup_time_hour']>=0) & (newFrame['pickup_time_hour'] < 6 )].index
  tt.loc[d0_index,'time_of_day']=0

  d1_index = newFrame[(newFrame['pickup_time_hour']>=6) & (newFrame['pickup_time_hour'] < 12 )].index
  tt.loc[d1_index,'time_of_day']=1

  d2_index = newFrame[(newFrame['pickup_time_hour']>=12) & (newFrame['pickup_time_hour'] < 18 )].index
  tt.loc[d2_index,'time_of_day']=2

  d3_index = newFrame[(newFrame['pickup_time_hour']>=18) & (newFrame['pickup_time_hour'] < 24 )].index
  tt.loc[d3_index,'time_of_day']=3

  return tt

def get_trip_distance(frame):
  newFrame = frame
  newFrame["distance"] = (((frame['pick_lat']-frame['drop_lat'])**2)+((frame['pick_lon']-frame['drop_lon'])**2)**0.5)
  return newFrame

def get_X_distance(frame):
  newFrame = frame
  newFrame["distance_X"] = ((frame['pick_lat']-frame['drop_lat']))
  return newFrame

def get_Y_distance(frame):
  newFrame = frame
  newFrame["distance_Y"] = ((frame['pick_lon']-frame['drop_lon']))
  return newFrame

def get_pick_XY(frame):
  newFrame = frame
  newFrame["pick_X"] = np.cos(frame['pick_lat'])* np.cos(frame['pick_lon'])
  newFrame["pick_Y"] = np.cos(frame['pick_lat'])* np.sin(frame['pick_lon'])
  return newFrame

def get_drop_XY(frame):
  newFrame = frame
  newFrame["drop_X"] = np.cos(frame['drop_lat'])* np.cos(frame['drop_lon'])
  newFrame["drop_Y"] = np.cos(frame['drop_lat'])* np.sin(frame['drop_lon'])
  return newFrame

In [130]:
cleaned_train_set = preprocess_train_data(dataframe)
labels = get_labels(dataframe)
cleaned_train_set.head()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019,11,1,0,20,2019,11,1,0,34,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019,11,1,0,56,2019,11,1,1,9,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019,11,1,1,8,2019,11,1,1,26,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019,11,1,2,27,2019,11,1,2,37,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,2019,11,1,5,38,2019,11,1,6,35,0.078384,0.21537,0.0320,-0.139842,-0.644356,-0.195765,-0.780623


In [0]:
cleaned_train_set = cleaned_train_set.drop('pickup_time_year',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_month',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_day',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_minute',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_year',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_month',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_day',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_minute',1)
#cleaned_train_set = set_time_of_the_day(cleaned_train_set)

In [126]:
cleaned_train_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_hour,drop_time_hour,distance,pick_X,pick_Y,drop_X,drop_Y
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,0,0.022663,-0.175519,-0.818213,-0.187363,-0.791950
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,1,0.006875,-0.173513,-0.805325,-0.174935,-0.788535
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1,1,0.050301,-0.197078,-0.786528,-0.154631,-0.778754
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2,2,0.007603,-0.175582,-0.781100,-0.169414,-0.781369
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,5,6,0.078384,-0.139842,-0.644356,-0.195765,-0.780623


In [0]:
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','pick_lat','pick_lon','drop_lat','drop_lon',
               'fare','pickup_time_day','pickup_time_hour','pickup_time_minute']
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','fare','pickup_time_hour','distance', 
                'distance_X', 'distance_Y']
#categorical_cols = ['time_of_day']

In [0]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    cleaned_train_set,
    labels,
    test_size=0.20,
    shuffle=True,
    stratify=labels,
    random_state=RANDOM_SEED
)

In [0]:
numeric_preprocessing_steps = Pipeline([
    ('minmax_scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])


preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        #("categorical", categorical_transformer, categorical_cols)
    ],
    remainder = "drop"
)

In [0]:
data_pipeline = Pipeline([
    ("preprocessor", preprocessor)
])

In [0]:
data_pipeline.fit(X_train)
d_train = data_pipeline.transform(X_train)
d_eval = data_pipeline.transform(X_eval)

In [151]:
d_train.shape

(13574, 10)

In [105]:
d_train[0]

array([2.13821228e-04, 0.00000000e+00, 2.06676026e-03, 8.20631136e-04,
       1.33297494e-03, 5.27573862e-03, 4.78260870e-01, 5.29539131e-07,
       9.89144463e-01, 1.12277126e-02, 6.50507521e-03, 1.75286308e-01,
       3.62912011e-03, 3.62912011e-03])

In [0]:
from sklearn.utils.class_weight import compute_sample_weight
sample_w = compute_sample_weight(class_weight={0:5, 1:1}, y=Y_train)

In [153]:
#drop lat lon all
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 1000, max_depth =8, subsample = 1.0)
gbclf.fit(d_train, Y_train, sample_w)
gbclf.score(d_eval, Y_eval)

0.9563936358279317

In [0]:
cleaned_test_data = preprocess_train_data(test)

cleaned_test_data = cleaned_test_data.drop('pickup_time_year',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_month',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_day',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_minute',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_year',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_month',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_day',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_minute',1)

test_set = data_pipeline.transform(cleaned_test_data)

In [155]:
test_set.shape

(8576, 10)

In [156]:
preds = gbclf.predict(test_set)
print(preds)

[1 1 1 ... 1 1 1]


In [157]:
preds.shape

(8576,)

In [158]:
output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output.head()

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


In [0]:
output.to_csv('/content/Ridefare_submission_10.csv',index=False)